*   Adjust the num_classes in the Dense layer based on the number of classes in your specific task.

*   Replace 'path_to_training_data' and 'path_to_validation_data' with the actual paths to your training and validation datasets.

This code uses the VGG16 model pre-trained on ImageNet, replaces the top layers with custom layers for the new task, freezes the pre-trained layers, and fine-tunes the model on a new dataset using the fit method. Adjust the optimizer, loss function, and other parameters based on your specific requirements.

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load pre-trained VGG16 model without top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for the new task
num_classes = 3
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Adjust num_classes for your specific task

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model on a new dataset
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(r'E:/ProcessedData/training_data', target_size=(224, 224), batch_size=32, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(r'E:/ProcessedData/validation_data', target_size=(224, 224), batch_size=32, class_mode='categorical')

model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=10, validation_data=validation_generator, validation_steps=len(validation_generator))


D:\Anaconda\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"





58889256/58889256 [==============================] - 16s 0us/step


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'E:/ProcessedData/training_data'